# Imports

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, "lib/")
from data.coco_dataset import CocoDataset
from utils.preprocess_sample import preprocess_sample
from utils.collate_custom import collate_custom
from utils.utils import to_cuda_variable
import utils.result_utils as result_utils
from utils.json_dataset_evaluator import evaluate_boxes
from model.detector import detector

torch_ver = torch.__version__[:3]

# Parameters

In [2]:
# Pretrained model
# https://s3-us-west-2.amazonaws.com/detectron/35857389/12_2017_baselines/e2e_faster_rcnn_R-50-FPN_2x.yaml.01_37_22.KSeq0b5q/output/train/coco_2014_train%3Acoco_2014_valminusminival/generalized_rcnn/model_final.pkl
arch='resnet50'
pretrained_model_file = 'files/trained_models/faster/e2e_faster_rcnn_R-50-FPN_2x.pkl'

# COCO minival2014 dataset path
coco_ann_file='datasets/data/coco/annotations/instances_minival2014.json'
img_dir='datasets/data/coco/val2014'

# Create dataset

In [3]:
dataset = CocoDataset(ann_file=coco_ann_file,img_dir=img_dir,
                       sample_transform=preprocess_sample(target_sizes=[800],fpn_on=True))
dataloader = DataLoader(dataset, batch_size=1, # only batch_size=1 is supported by now
                        shuffle=False, num_workers=0, collate_fn=collate_custom)

loading annotations into memory...
Done (t=0.84s)
creating index...
index created!


# Create detector model

In [4]:
model = detector(arch=arch,
                 detector_pkl_file=pretrained_model_file,
                 conv_body_layers=['conv1','bn1','relu','maxpool','layer1','layer2','layer3','layer4'],
                 conv_head_layers='two_layer_mlp',
                 fpn_layers=['layer1','layer2','layer3','layer4'],
                 fpn_extra_lvl=True,
                 roi_height=7,
                 roi_width=7,
                 roi_spatial_scale=[0.25,0.125,0.0625,0.03125],
                 roi_sampling_ratio=2,
                 use_rpn_head = True)
model = model.cuda()

Loading pretrained weights:
-> loading conv. body weights
-> loading output head weights
-> loading rpn head weights
-> loading FPN lateral weights
-> loading two layer mlp conv head...


# Evaluate

In [5]:
# Create data structure to store results
all_boxes, all_segms, all_keyps = result_utils.empty_results(dataset.num_classes, len(dataset)) 
# (only all_boxes will be used for fast RCNN)

In [6]:
# Compute detections for whole dataset
for i, batch in enumerate(dataloader):
    batch = to_cuda_variable(batch)
    # forward pass
    if torch_ver=="0.4": # handle change in "volatile"
        with torch.no_grad():
            class_scores,bbox_deltas,rois,_=model(batch['image'],
                                                  scaling_factor=batch['scaling_factors'])   
    else:
        class_scores,bbox_deltas,rois,_=model(batch['image'],
                                              scaling_factor=batch['scaling_factors'])   
    # postprocess output:
    # - convert coordinates back to original image size, 
    # - treshold proposals based on score,
    # - do NMS.
    scores_final, boxes_final, boxes_per_class = result_utils.postprocess_output(rois,
                                                                    batch['scaling_factors'],
                                                                    batch['original_im_size'],
                                                                    class_scores,
                                                                    bbox_deltas)
    # store results
    result_utils.extend_results(i, all_boxes, boxes_per_class)
    
    if i%100==0:
        print("{}/{}".format(i+1,len(dataset)))
        
print('Done!')

1/5000
101/5000
201/5000
301/5000
401/5000
501/5000
601/5000
701/5000
801/5000
901/5000
1001/5000
1101/5000
1201/5000
1301/5000
1401/5000
1501/5000
1601/5000
1701/5000
1801/5000
1901/5000
2001/5000
2101/5000
2201/5000
2301/5000
2401/5000
2501/5000
2601/5000
2701/5000
2801/5000
2901/5000
3001/5000
3101/5000
3201/5000
3301/5000
3401/5000
3501/5000
3601/5000
3701/5000
3801/5000
3901/5000
4001/5000
4101/5000
4201/5000
4301/5000
4401/5000
4501/5000
4601/5000
4701/5000
4801/5000
4901/5000
Done!


In [7]:
# Save detection results
np.save('files/results/all_boxes_faster.npy',all_boxes)

In [8]:
# Compute evaluation metrics
coco_eval = evaluate_boxes(json_dataset=dataset.coco, 
                           all_boxes=all_boxes, 
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=1.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=39.80s).
Accumulating evaluation results...
DONE (t=6.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.379
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.499
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.311
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.510
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1